<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/Data/Trigger_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 가져오기 및 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
class Data:
  def __init__(self, file_path):
    self.dataFrame = pd.read_csv(file_path, encoding='cp949')
  
  def get_content_by_col(self, colomn):
    # 같은 내용의 카테고리 합치기
    self.dataFrame[colomn] = self.dataFrame[colomn].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)
    content = list(set(self.dataFrame[colomn].values.tolist()))
    content = sorted(content)
    return content

### 라이브러리 설치

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install customized_konlpy # 형태소 사용자 사전 추가
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,415 kB]
Get:13 http://archive.

키워드 추출기

In [ ]:
!pip3 install krwordrank # 키워드 추출기
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize


def keyword_tracker_KR(text_group): 
  texts = [normalize(text, english=True, number=True) for text in text_group]  # 문장내 특수문자 제거하기
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )
  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10
  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
  keyword = []
  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
      keyword.append(word)
  return keyword






#형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt

okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls

def keyword_tracker_Okt(text_group): # memo-문장이 들어있는 리스트
  text_group = [ re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',text) for text in text_group]  # 문장내 특수문자 제거하기
  text_group = ' '.join(text_group)
  text_group=morph_and_stopword(text_group) # 형태소 분리
  text_group=Counter(text_group)
  text_group_rank=text_group.most_common(5) # 상위 5개
  return text_group_rank


In [ ]:
# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

# 업무 구조 - 4단계 구획하기

In [ ]:
from numpy import nan

class Main_divider:
  def __init__(self):

    self.factor_trigger = re.compile(r'고장[\s]*부위|문의[\s]*내용|증상[\s]*상세|문의[\s]*사항|문의[\s]*구분')
    self.factor_check = re.compile(r'확인[\s]*[내용|처리[\s]*내용]*|확인[\s]*사항') 
    self.factor_figure = re.compile(r'안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항') 
    self.factor_promise = re.compile(r'약속[\s]*[내용|사항]') 
    # action부분에서 이러한 단어가 있으면 중요한 문장으로 생각한다.
    self.actor = 'ABS|ADAS|AUX|DPF|ESC|GPS|ODO|OTP|RPM|SD|TCS|USB|가리개|가스|각도|강제|개방|거치대|게이지|경고등|'\
    '경보|경적|계기판|고장|고정|공기압|공조|교체|교환|구리스|구토|그리스|글로브|급발진|기름|기어|기억|깜빡|꺼|꼽|나사|'\
    '내비게이션|내장제|냄새|네트[\s]*워크|녹이|누유|단말기|단차|닫|담배|담뱃재|동물 털|동파|드럼|들리|등받이|디스크|따뜻|떨리|떨림|떨어|'\
    '라우터|라이트|레버|레이더|마감|마모|마스터키|만차|매트|모드|미러|미점등|밀림|바디|바람[\s]*빠|바퀴|박스|받침대|발판|방전|방지|방향|배선|'\
    '배출|배치|배터리|버튼|범퍼|변속|보관함|보닛|보조|보충|볼륨|부족|부착|부팅|분실|불량|불빛|브레이크|블랙박스|블루투스|비치함|'\
    '뽑|사고조사|사이드|삽입|선루프|선바이저|선팅|세차|센서|소거|소리|소음|손상|손잡이|수동|스마트[\s]*키|스캐[너|서]|스크래치|스티커|스패너|습기|'\
    '시거잭|시동|시야|시트|신호|실내등|실행|쏠|악취|안나|안내[\s]*[약속[\s]*내용|사항]|안전벨트|알림|언|얼라이먼트|얼어|업그레이드|'\
    '업데이트|에어|에이필러|엑셀|엔 와 향|엔진|연결|연동|연료|열|열선|오디오|오류|오일|옥스 트라|온도|와이파이|와이퍼|외관|요소수|운전석|워셔액|워시|'\
    '원격|위생|위치|유리|유심|음성|이동|인식|인터넷|입차|잠금|장착|장치|재시작|전구|전원|전조등|점등|점프|제동등|제어|조수석|조절|좌석|주유|주행등|'\
    '중계기|진단|찢|차 세 제어|차가운|찬|창문|처리[\s]*[내용|사항]|청소|체크|초기화|출차|충돌|충전|카드|카메라|캡|커넥터|컨트롤|컴파운드|코드절상|클리닝|'\
    '키 아웃|키 인식|타는|타이어|탈거|탐색|태깅|테스트|테이프|통신|트렁크|티씨에스|파손|파스|판단|패드|펑크|편의|포켓|포트|품목|퓨즈|플레이트|하이패스|핸들|'\
    '혼유|화면|환경이슈|후미등|후방|후진|훼손|휠|히터'
    self.actor_figure = re.compile(self.actor) 
    self.service_info ='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*|(관제\s예약\s)*URL[주소:\s]*[^\n]+|'\
    '(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*|([원]*예약번호|블락번호|고객운행)[^\n]*|'\
    '((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*|'\
    '([\\n]*(채팅[\s]*상담|톡|요청자)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인/요청자 제거
    self.semi_divider = re.compile('진행|확인|안내|가능|요청|발송|불가|적용|해결|문자|조정|수긍|과실|양해|는지|부탁|도움|도와|부과')


  def chunck_maker(self, memo):
    memo = re.sub(r'#|_', ' ', memo.upper())
    memo = re.sub(r'\([^\)\()]*\)|([0-9]+/[0-9]+)', '', memo) # 삭제: 괄호 내용 & 날짜 
    memo = re.sub(r'[ㄴ|ㅁ|ㅡ|\-|(\t)|:|\|]', '\n', memo) # 문장 구획하기
    memo = re.sub(r'( )*(\n+)( )*', '\n', memo)
    memo = re.sub(r'(\n)+', '\n', memo)
    memo = re.sub(r'( )+', ' ', memo)
    return memo
    


  def track_Trigger(self, memo):
    # 1....... 주어진 문장을 \n로 분할하여 리스트만들기  
    memo = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\(\)\s]', '', memo)
    memo = re.sub(self.service_info, '', memo)
    memo = self.chunck_maker(memo)
    memo_chunck = memo.split('\n')

    # 2............   필요한 리스트 초기화
    flag = 'None'
    figure=[]
    check=[]
    trigger=[]
    semi_trigger = True

    # 3.................... 쪼개진 문장에서 Trigger / 확인사항/ figure / 나누기 
    for i in memo_chunck:
        if self.factor_trigger.search(i):
          flag = 'trigger_mode'
          continue
        elif self.factor_check.search(i): 
          flag = 'check_mode'
          continue
        elif self.factor_figure.search(i): 
          flag = 'figure_mode'
          continue
        elif self.factor_promise.search(i): 
          flag = 'promise_mode'   

        if flag == 'trigger_mode': trigger.append(i)
        elif flag == 'check_mode': check.append(i)
        elif flag == 'figure_mode':
          if self.actor_figure.search(i): 
            if self.semi_divider.search(i): semi_trigger = False
            trigger.append(i) if semi_trigger else figure.append(i)
        elif flag == 'promise_mode':  pass

    # 4........ 결과 출력하기
    print('▷ Chunk: ', memo_chunck)
    if len(trigger) != 0: print("증상상세:", trigger)
    #if len(check)  != 0: print("확인내용:", check)
    if len(figure) != 0: print("안내처리:", figure)
    return figure, check, trigger 





  def refine_action(self, action_list):
    action_list = [x for x in action_list if x is not nan]
    action_list = "\n".join(action_list)
    action_list = re.sub(r'[네|내]비[게이션]*', '내비게이션', action_list.upper())
    action_list = re.sub(r'본넷|본네트|본닛|보닛트', '보닛', action_list)
    action_list = re.sub(r'라이트/전구', '라이트 및 전구', action_list)
    action_list = re.sub(r'문제어|도어( )*제어', '문 제어', action_list)
    action_list = re.sub(r'삽입', '장착', action_list)
    action_list = re.sub(r'교환', '교체', action_list)
    action_list = re.sub(r'긴출', '긴급출동', action_list)
    action_list = re.sub(r'블박', '블랙박스', action_list)
    action_list = re.sub(r'엔[.|,]*와[.|,]*향', '엔와향', action_list)
    action_list = re.sub(r'(좌[ /.,]*우)|(전[ /.,]후)|(A/S)|(\d[\s]*본|\d[\s]*짝|\d[\s]*EA)', '', action_list)
    action_list = re.sub(r'(전|앞|뒤|좌|우|전륜|후륜|하향|상향|하부|상부|내부|외부|좌측|우측| )+ *({})'.format(self.actor), r' \2', action_list) # 후지움
    action_list = self.chunck_maker(action_list)
    action_list = re.split('\n|/|\.|,', action_list) # 문장분리
    
    real_action_list = []
    no_use = re.compile(r'->|[0-9]{3,10}|@|가온|담당자') 
    for i in action_list:
      i = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',i)
      try: i = correction(i.strip()) # 맞춤법 교정하기(네이버 검사기)
      except: pass
      i = re.sub(r'( )+등', r'등', i)
      if self.actor_figure.search(i):
        if not no_use.search(i): #완벽하게 통과된 
          real_action_list.append(i)
    return real_action_list

In [ ]:
file_path = '/content/drive/MyDrive/2-2._lesion_info_table.csv'
data = Data(file_path)
inspect_type = data.get_content_by_col("lesion_detail_type")
divider = Main_divider()

'하이패스_카드오류'

In [ ]:
f_trigger = open('/content/drive/MyDrive/Trigger_list_final.txt','w')
f_action = open("/content/drive/MyDrive/action_list_final.csv", "w")
f_action.write( 'inspect_type' + ',' + 'content'+ '\n')


dic_trigger={}
dic_check={}
dic_figure={}
dic_action={}
inspect_num=0
for inspect in inspect_type: # 특정 카테고리
  all_case_trigger = []
  all_case_check = []
  all_case_figure = []
  all_case_action = []

  id_list = data.dataFrame.groupby('lesion_detail_type').get_group(inspect) 
  id_list = set(id_list['lesion_id'].values.tolist())
  for id in id_list: # 카테고리 내의 사건번호
    case = data.dataFrame.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    case = case.drop_duplicates(['memo']) # memo의 중복행 제거
    content = case['memo'].values.tolist()
    print("============================================\n{}▶ Origin: ".format(inspect_type[inspect_num]), content)

    # 다음 원소에도 Trigger가 존재하는가 
    point = 0
    for i in range(0, len(content)):
        if divider.factor_trigger.search(str(content[i])):
          figure, check, trigger = divider.track_Trigger(content[i]) # 로직 분석기 돌리기
          all_case_trigger += trigger
          all_case_check += check
          all_case_figure += figure
          point = i
    else: 
          action = content[point+1:]
          action = divider.refine_action(action) # Action 중에서 필요한 정보만 추출하기
          print('차후수리: ', action)
          all_case_action += action


  dic_trigger[inspect_num]= all_case_trigger
  dic_check[inspect_num]= all_case_check
  dic_figure[inspect_num]= all_case_figure
  dic_action[inspect_num]= all_case_action

  for i in sorted(all_case_action): # action 데이터 저장
    f_action.write(inspect_type[inspect_num] + ',' + i + '\n')
  for i in sorted(all_case_trigger): # trigger 데이터 저장
    f_trigger.write(str(inspect_num)+'\t'+ i +'\t'+str(inspect_num)+'\n') 

  print('\n\n*****************************************************************************************************************')
  print(" >> ", inspect_type[inspect_num])
  print(all_case_trigger)
  print(all_case_check)
  print(all_case_figure)
  print(all_case_action)
  print('*****************************************************************************************************************\n\n')


  inspect_num += 1



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
차후수리:  []
타이어_파스/펑크(전좌)▶ Origin:  ['전좌/우타이어마모로교체필요\nㄴ가온오토 화성점 입고 \n\n- 선제조치장애', '신규접수->처리중 (2020-12-29 09:20:05)', '(화성점)145허5341/더뉴아반떼/34074Km/전 좌우 타이어(195/65/15,넥센타이어,iq시리즈1,화성점장착)교체 완료 ', '처리중->조치완료 (2020-12-29 11:46:25)', '조치완료->검수중 (2020-12-29 11:46:30)', '검수중->검수완료 (2020-12-29 11:46:35)']
차후수리:  ['타이어 교체 완료']
타이어_파스/펑크(전좌)▶ Origin:  ['원예약번호 : 34877075\n1. 타이어 펑크 : (O)\n2. 타이어 파스 : (X)\n3. 공기압경고등 점등여부 : (X)\n4. 증상 상세내용 : \n\n$고객_운행중\nㅁ문의내용\nㄴ운행전 타이어 펑크로 인입 \n\nㅁ확인 내용\nㄴ본인확인 \n\nㅁ안내/처리 내용\nㄴ운전석 앞바퀴라고 하심 \nㄴ사진확인 완료\nㄴ인근 쏘카존 상급차량 변경 안내 / 대차존 반납 수긍  \nㄴ대여료, 보험료, 주행요금은 동일하게 적용되며 사고시 휴차료와 하이패스 및 연장하셨을 경우의 연장요금은 대차차량으로 적용됨. 도보이동 및 지연시간 30분 무상연장 안내 후 대차진행.\nㄴ수긍 종료\n\nㅁ약속 내용\nㄴX', '12/4 가온오토 남양주 방문예정', '신규접수->처리중 (2020-12-04 09:05:47)', '운전석 전면 타이어 공기압 보충완료 육안으로 파스및 펑크 확인되지않습니다', '처리중->조치완료 (2020-12-04 14:04:35)']
▷ Chunk:  ['', '타이어 펑크', '타이어 파스', '공기압경고등 점등여부', '증상 상세내용', '문의내용', '운행전 타이어 펑크로 인입', '확인 내용', '안내처리 내용', '운전석 앞바퀴라고 하심', '사진확인 완료', '인근 쏘카존 